In [1]:
from mapping_reef_classes import get_coordinates
import os
import re
import pandas as pd
import numpy as np
import datetime
import pickle
import shutil
from src import list_files, copy_imgs_2_drive, return_time

In [2]:
tif_files = list_files(r"C:\Users\ageglio\OneDrive - DOI\Documents - Reef Mapping\Data Release\NLM_DataRelease", 'tif')
# further sorting of file search
tif_files = [re.findall('.*_BY_0.5m.tif', i) for i in tif_files] + [re.findall('.*0.5m_BY.tif', i) for i in tif_files]
# tif_files = [re.findall('.*Classified.*', i) for i in tif_files]
tif_files = list(set([item for sublist in tif_files for item in sublist]))
tif_files.sort()
print(len(tif_files))

68


In [3]:
# 7, 8, 9
reef_dict = {"BH": "Bay Harbor", "CH": "Cathead Point", "ER": "Elk Rapids", "FI": "Fishermans Island", "IP":"Ingalls Point",
             "LR": "Lees Reef", "MS": "Manistique Shoal", "MP": "Mission Point", "MR":"Mudlake Reef", "NPNE": "Northport Bay NE",
             "NPNW":"Northport Bay NW", "NPS":"Northport Bay S", "NP": "Northport Point", "SP": "Suttons Point", 
             "TC":"Tannery Creek", "TS":"Traverse Shoal", "TP":"Tuckers Point", "WP": "Wiggins Point"}
df_tiffs = get_coordinates().return_min_max_tif_df(tif_files)
df_tiffs = df_tiffs.drop_duplicates(subset="reef").reset_index(drop=True)
df_tiffs["Reef_Name"] = df_tiffs.reef.apply(lambda x: reef_dict[x])
# df_tiffs.to_csv("reef_coordinates.csv")
df_tiffs # 18 reefs

c:\Users\ageglio\AppData\Local\miniconda3\envs\img_processing\Lib\site-packages\osgeo\gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


,filename,min_lat,min_lon,max_lat,max_lon,avr_lat,avr_lon,reef,Reef_Name
0,BH_BY_0.5m.tif,45.370376,-85.000113,45.378667,-84.980203,45.374522,-84.990158,BH,Bay Harbor
1,CH_BY_0.5m.tif,45.180576,-85.637792,45.204111,-85.583445,45.192344,-85.610619,CH,Cathead Point
2,ER_BY_0.5m.tif,44.895292,-85.429009,44.916075,-85.404111,44.905684,-85.416560,ER,Elk Rapids
3,FI_BY_0.5m.tif,45.286375,-85.372273,45.303263,-85.344850,45.294819,-85.358561,FI,Fishermans Island
4,IP_BY_0.5m.tif,45.067733,-85.577850,45.077351,-85.563789,45.072542,-85.570819,IP,Ingalls Point
5,LR_0.5m_BY.tif,44.885100,-85.616915,44.919480,-85.601494,44.902290,-85.609204,LR,Lees Reef
6,MS_0.5m_BY.tif,45.888421,-86.251183,45.905580,-86.232800,45.897000,-86.241992,MS,Manistique Shoal
7,MP_BY_0.5m.tif,44.984333,-85.505051,45.035714,-85.469587,45.010023,-85.487319,MP,Mission Point
8,MR_BY_0.5m.tif,44.964443,-85.400412,44.997639,-85.384873,44.981041,-85.392642,MR,Mudlake Reef
9,NPNE_BY_0.5m.tif,45.126868,-85.584189,45.148469,-85.560859,45.137669,-85.572524,NPNE,Northport Bay NE


In [4]:
survey_names = pd.read_excel(r"Z:\Database\OP_TABLE.xlsx")
df_collects = survey_names[["COLLECT ID", "OP_DATE", "SITE NAME", "SURVEY NAME", "LATTITUDE", "LONGITUDE"]]
print(df_collects.shape) # (608, 6)
reef_overlap_df = get_coordinates.reef_overlap(df_tiffs, df_collects).drop_duplicates(subset="COLLECT ID", keep="last")
# reef_overlap_df = reef_overlap_df[reef_overlap_df.OP_DATE>"2021-05-01"]
# reef_overlap_df.to_csv("Reef_tif_collects_overlap.csv")
reef_overlap_df.shape # (608, 6)

(608, 6)


(68, 7)

In [6]:
collect_list = reef_overlap_df["COLLECT ID"]
df_unp20 = pd.read_csv(r"Z:\Database\02__MetadataCombined\all_unpacked_images_metadata_2020.csv", index_col=0, low_memory=False)
df_unp20 = df_unp20[df_unp20.collect_id_y.isin(collect_list)]
df_unp21 = pd.read_csv(r"Z:\Database\02__MetadataCombined\all_unpacked_images_metadata_2021.csv", index_col=0, low_memory=False)
df_unp21 = df_unp21[df_unp21.collect_id_y.isin(collect_list)]
df_unp22 = pd.read_csv(r"Z:\Database\02__MetadataCombined\all_unpacked_images_metadata_2022.csv", index_col=0, low_memory=False)
df_unp22 = df_unp22[df_unp22.collect_id_y.isin(collect_list)]
df_unp23 = pd.read_csv(r"Z:\Database\02__MetadataCombined\all_unpacked_images_metadata_2023.csv", index_col=0, low_memory=False)
df_unp23 = df_unp23[df_unp23.collect_id_y.isin(collect_list)]
df_unp_u = pd.concat([df_unp20, df_unp21, df_unp22, df_unp23])
df_unp_u = df_unp_u[df_unp_u.Usability == "Usable"]
col_to_use = ['Time_s', 'filename', 'Latitude', 'Longitude', 'DepthFromSurface_m', 'DistanceToBottom_m', 'TotalWaterColumn_m', 'Speed_kn', 'Time_UTC',
              'DataSource', 'PointCloudStandardDeviation', 'PointCloudElevation_m', 'DateGPS', 'SpeedGPS_kn', 'HeadingGPS_deg','Temperature_C', 'Salinity_ppt',
              'SoundSpeed_mps', 'LatitudeINS', 'LongitudeINS', 'LatitudeGPS', 'LongitudeGPS', 'HeadingINS_deg', 'PitchINS_deg', 'RollINS_deg', 'XspeedINS_mps',
              'YspeedINS_mps', 'LogFile', 'MissionName', 'VehicleID', 'SRPMissionActive', 'LatitudeGPSDir', 'LongitudeGPSDir','Usability', 'collect_id_x',
              'CollectID', 'BagFile', 'InvalidTelnetLines', 'image_path', 'collect_id_y','UsingINS', 'GPSvalid', 'INSvalid', 'REEF']
df_unp_u = pd.merge(df_unp_u, reef_overlap_df[["COLLECT ID", "REEF"]], left_on="collect_id_y", right_on = "COLLECT ID")
df_unp_u = df_unp_u[col_to_use]
df_unp_u["Year"] = df_unp_u.Time_s.apply(lambda x: return_time().get_Y(x))
df_unp_u["Month"] = df_unp_u.Time_s.apply(lambda x: return_time().get_m(x))
df_unp_u["Day"] = df_unp_u.Time_s.apply(lambda x: return_time().get_d(x))
df_unp_u["Time"] = df_unp_u.Time_s.apply(lambda x: return_time().get_t(x))
df_unp_u.to_csv(r"C:\Users\ageglio\ageglio-1\mussel-finder\reef-overlap_metadata.csv")
print(df_unp_u.shape) # (26846, 48)

(26846, 48)


In [7]:
df_unp_u = pd.read_csv(r"C:\Users\ageglio\ageglio-1\mussel-finder\reef-overlap_metadata.csv", index_col=0)
df_unp_u.head()
# copy_imgs_2_drive(df_unp_u, dest_folder = r"H:\reef-collect-overlap")

,Time_s,filename,Latitude,Longitude,DepthFromSurface_m,DistanceToBottom_m,TotalWaterColumn_m,Speed_kn,Time_UTC,DataSource,...,image_path,collect_id_y,UsingINS,GPSvalid,INSvalid,REEF,Year,Month,Day,Time
0,1.597397e+09,PI_1597397145_474.png,45.185024,-85.632666,3.775571,2.510704,6.286275,2.089296,20:25.5,log - point cloud matched,...,Z:\__Organized_Directories_InProgress\2020_Unp...,20200814_001_Iver3069_ABS1,NaN,NaN,NaN,CH,2020,8,14,05:25:45
1,1.597397e+09,PI_1597397216_223.png,45.185450,-85.632166,4.424786,2.878462,7.303248,1.472393,21:36.2,log - point cloud matched,...,Z:\__Organized_Directories_InProgress\2020_Unp...,20200814_001_Iver3069_ABS1,NaN,NaN,NaN,CH,2020,8,14,05:26:56
2,1.597397e+09,PI_1597397220_241.png,45.185469,-85.632133,4.534856,2.515431,7.050287,1.470000,21:40.2,log - point cloud matched,...,Z:\__Organized_Directories_InProgress\2020_Unp...,20200814_001_Iver3069_ABS1,NaN,NaN,NaN,CH,2020,8,14,05:27:00
3,1.597397e+09,PI_1597397224_261.png,45.185488,-85.632103,4.573502,2.428986,7.002488,1.480000,21:44.3,log - point cloud matched,...,Z:\__Organized_Directories_InProgress\2020_Unp...,20200814_001_Iver3069_ABS1,NaN,NaN,NaN,CH,2020,8,14,05:27:04
4,1.597397e+09,PI_1597397228_602.png,45.185508,-85.632068,4.600000,2.799527,7.399527,1.479527,21:48.6,log - point cloud matched,...,Z:\__Organized_Directories_InProgress\2020_Unp...,20200814_001_Iver3069_ABS1,NaN,NaN,NaN,CH,2020,8,14,05:27:09


In [ ]:
time_s = df_unp_u.loc[0, 'Time_s']
def get_time_obj(time_s):
    return datetime.datetime.fromtimestamp(time_s)
def get_Y(time_s):
    return get_time_obj(time_s).strftime('%Y') 
def get_m(time_s):
    return get_time_obj(time_s).strftime('%m')
def get_d(time_s):
    return get_time_obj(time_s).strftime('%d')
def get_t(time_s):
    return get_time_obj(time_s).strftime('%H:%M:%S')

In [ ]:
# import matplotlib.pyplot as plt
# from osgeo import gdal, osr
# ds = gdal.Open('DEM\Huron1_Albers_3m_NAVDm.img')
# data = ds.ReadAsArray()
# gt = ds.GetGeoTransform()   
# proj = ds.GetProjection()
# xres = gt[1]
# yres = gt[5]

# xmin = gt[0] + xres * 0.5
# xmax = gt[0] + (xres * ds.RasterXSize) - xres * 0.5
# ymin = gt[3] + (yres * ds.RasterYSize) + yres * 0.5
# ymax = gt[3] - yres * 0.5

# x_center=(xmin+xmax)/2
# y_center=(ymin+ymax)/2    
# #################################################################################
# ds.RasterXSize
# import pandas as pd
# pd.DataFrame(gt)